In [1]:
import os
import time
from datetime import timedelta

import nbformat
import numpy as np
from nbconvert.preprocessors import ExecutePreprocessor

In [2]:
### TO HIDE THE LOGGING FROM SPARK ###

from IPython.display import HTML

HTML('''<script>
var code_show_err = false;
var code_toggle_err = function() {
    var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
    var stderr = Array.from(stderrNodes)
    if (code_show_err){
        stderr.forEach(ele => ele.style.display = 'block');
    } else {
        stderr.forEach(ele => ele.style.display = 'none');
    }
    code_show_err = !code_show_err
}
document.addEventListener('DOMContentLoaded', code_toggle_err);
</script>
To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [3]:
filename = "2_model.ipynb"
with open(filename) as ff:
    nb_in = nbformat.read(ff, nbformat.NO_CONVERT)

In [ ]:
f1_scores = []
start = time.time()
for seed in [10, 20, 30, 40, 50]:
    executor = ExecutePreprocessor(timeout=12*60*60, kernel_name="python3")
    os.environ["EXSTRAQT_SEED"] = str(seed)
    nb_out = executor.preprocess(nb_in)
    last_block_output = nb_out[0]["cells"][-1]["outputs"][0]["text"]
    last_block_output_lines = last_block_output.split("\n")
    print(last_block_output_lines[0])
    delta = round(time.time() - start)
    print(f"Elasped time: {timedelta(seconds=delta)}")
    start = time.time()
    print()
    f1_scores.append(float(last_block_output_lines[1]))

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/25 18:57:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "ERROR".
                                                                                

In [ ]:
f1_final = round(max(f1_scores), 2)
f1_std = round(np.std(f1_scores), 2)

print(f"{f1_final} ± {f1_std}")